In [1]:
import json
import geopandas as gpd
from shapely import Polygon

# Constants
PATH = '../data/supply/lots.gpkg'

cargo = gpd.read_file(PATH, layer='cargo', columns=['pkid', 'roadname'])
cargo['centroid'] = cargo.centroid
realtime = gpd.read_file('../data/realtime-lot.geojson', columns=['id', 'name']).to_crs(3826)

cargo_lots = gpd.sjoin(cargo, realtime, how='inner', predicate='intersects')
cargo_lots['idLeft'] = cargo_lots.apply(lambda x: f'{x["roadname"]}_{x["pkid"]}', axis=1)
cargo_lots = cargo_lots[cargo_lots['idLeft'] == cargo_lots['id']][['id', 'centroid']].rename(columns={'centroid': 'geometry'}).reset_index(drop=True)
display(cargo_lots)

,id,geometry
0,漢口街2段_12,POINT (300839.864 2771015.699)
1,長安西路B_34,POINT (301228.538 2771668.416)
2,北安路458巷41弄_69,POINT (305056.347 2774614.496)
3,江南街_04,POINT (308302.088 2774510.741)
4,民權東路6段B_04,POINT (309026.185 2773522.547)
...,...,...
1218,公舘路423巷_10,POINT (300727.196 2779233.784)
1219,承德路二段A_06,POINT (302295.51 2772071.349)
1220,南寧路_01,POINT (300902.076 2769895.004)
1221,重慶北路3段236巷45弄_10,POINT (301685.513 2773712.314)


In [2]:
status: dict

with open('../data/history.json', 'r', encoding='utf-8') as f:
    status = json.load(f)

def count_non_nans(nums: list):
    return len([*filter(lambda x: x is not None, nums)])

def get_section_data(status_list: list, start: int, end: int):
    agg = []
    for i in range(7):
        agg.extend(status_list[start+i*144 : end+i*144])
    return agg

def get_sum(nums: list):
    return sum([num for num in nums if isinstance(num, float) or isinstance(num, int)])

def get_vacancy(id: str):
    lot = status.get(id, None)
    if isinstance(lot, dict):
        stat = [*lot['1'], *lot['2'], *lot['3'], *lot['4'], *lot['5'], *lot['6'], *lot['7']]
        if stat.count(None) > 150:
            return None
        else:
            morning = get_section_data(stat, 24, 72)        # Morning 04-12
            afternoon = get_section_data(stat, 72, 120)     # Afternoon 12-20
            night = get_section_data(stat, 120, 144)        # Night 20-04
            night.extend(get_section_data(stat, 0, 24))

            return (
                    round(1 - get_sum(morning) / count_non_nans(morning), 4),
                    round(1 - get_sum(afternoon) / count_non_nans(afternoon), 4),
                    round(1 - get_sum(night) / count_non_nans(night), 4)
                   )
    else:
        return None

vacancy = cargo_lots.apply(lambda x: get_vacancy(x['id']), axis=1, result_type='expand').rename(columns={0: 'Morning', 1: 'Afternoon', 2: 'Night'})
cargo_lots = gpd.GeoDataFrame(cargo_lots.join(vacancy)[['id', 'Morning', 'Afternoon', 'Night', 'geometry']])
cargo_lots.geometry = cargo_lots.buffer(50)
display(cargo_lots)

,id,Morning,Afternoon,Night,geometry
0,漢口街2段_12,0.5709,0.1378,0.2483,"POLYGON ((300889.864 2771015.699, 300889.623 2..."
1,長安西路B_34,0.6047,0.6042,0.3113,"POLYGON ((301278.538 2771668.416, 301278.297 2..."
2,北安路458巷41弄_69,0.6047,0.6431,0.5364,"POLYGON ((305106.347 2774614.496, 305106.106 2..."
3,江南街_04,0.1791,0.2359,0.0695,"POLYGON ((308352.088 2774510.741, 308351.847 2..."
4,民權東路6段B_04,0.6858,0.5880,0.3642,"POLYGON ((309076.185 2773522.547, 309075.944 2..."
...,...,...,...,...,...
1218,公舘路423巷_10,0.2872,0.2561,0.1056,"POLYGON ((300777.196 2779233.784, 300776.955 2..."
1219,承德路二段A_06,0.0034,0.0000,0.0099,"POLYGON ((302345.51 2772071.349, 302345.269 27..."
1220,南寧路_01,0.7095,0.7805,0.5927,"POLYGON ((300952.076 2769895.004, 300951.836 2..."
1221,重慶北路3段236巷45弄_10,0.0372,0.1484,0.0033,"POLYGON ((301735.513 2773712.314, 301735.272 2..."


In [3]:
grids = gpd.read_file('../data/location/grid.geojson',
                      columns=['Index', '批發', '零售', '郵政', '餐飲', '供給', 'geometry']).to_crs(3826)

def get_intersection_mean(grid: Polygon):
    buffers = cargo_lots[cargo_lots.intersects(grid)]
    return round(buffers['Morning'].mean(), 4), round(buffers['Afternoon'].mean(), 4), round(buffers['Night'].mean(), 4)

grids_vacancy = grids.apply(lambda x: get_intersection_mean(x['geometry']), axis=1, result_type='expand').rename(columns={0: '早', 1: '中', 2: '晚'})
grids = grids.join(grids_vacancy)
grids.to_crs(4326).to_file('../data/location/grid.geojson', index=False)

display(grids)

,Index,供給,批發,郵政,零售,餐飲,geometry,早,中,晚
0,1,0.0,0.0,0.0,0.0,0.1,"POLYGON ((296621.075 2779105.899, 296821.075 2...",NaN,NaN,NaN
1,2,0.0,0.0,0.0,0.0,4.1,"POLYGON ((296621.075 2778905.899, 296821.075 2...",NaN,NaN,NaN
2,3,0.0,0.0,0.0,0.0,0.2,"POLYGON ((296821.075 2780505.899, 297021.075 2...",NaN,NaN,NaN
3,4,0.0,0.2,0.0,0.4,1.6,"POLYGON ((296821.075 2780305.899, 297021.075 2...",NaN,NaN,NaN
4,5,0.0,0.0,0.0,0.2,0.0,"POLYGON ((296821.075 2780105.899, 297021.075 2...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2738,2739,2.1,1.0,0.0,2.5,4.7,"POLYGON ((312621.075 2770505.899, 312821.075 2...",NaN,NaN,NaN
2739,2740,13.9,4.0,0.0,6.1,8.7,"POLYGON ((312621.075 2770305.899, 312821.075 2...",NaN,NaN,NaN
2740,2741,0.0,1.4,0.0,1.4,0.4,"POLYGON ((312621.075 2770105.899, 312821.075 2...",NaN,NaN,NaN
2741,2742,0.0,0.7,0.0,0.9,0.4,"POLYGON ((312821.075 2770305.899, 313021.075 2...",NaN,NaN,NaN
